In [12]:
import bs4
import requests
import time
from multiprocessing import Pool

# para Session()
# from requests.adapters import HTTPAdapter
# from requests.packages.urllib3.util.retry import Retry


visitedLinks = set()

def get_links(start):
    links = set()
    links.add(start) 
     
    res = ''
    while res == '':
        try:
            res = requests.get(start)
            break
        except:
            time.sleep(1)
            continue
            
# #     outra opcao   
#     session = requests.Session()
#     retry = Retry(total=1000, connect=1000, read=1000, redirect=1000, backoff_factor=0.5)
#     adapter = HTTPAdapter(max_retries=retry)
#     session.mount(start, adapter)
#     session.mount(start, adapter)
#     res = session.get(start)
    
    soup = bs4.BeautifulSoup(res.text, 'lxml')     
    
    for link in soup.find_all('a', href=True):
        href = link["href"]
    
        # apenas links do site
        if (href.startswith(start) | href.startswith("/")):
            
            # trata urls relativos
            if href.startswith("/"):            
                href = start + href
                
            links.add(href)
    
    # remove todos os links que sao iguais a buscas passadas
    links = links.difference(visitedLinks)
    
    # armazena todos os links já consultados sem repeticoes
    visitedLinks.update(links)
    
    return list(links)
    

def crawl_bsf(start): 
    queue = []
    frontier = [start] # start -> frontier 
    count = 0
    
    while(len(frontier) != 0 and count < 1000):  
        seen = frontier.pop(0)        
        queue.append(seen)
        if (len(frontier) <= 1000):
            time.sleep(10)
            frontier.extend(get_links(seen)) # adiciona os vizinhos no fim da lista
        count += 1

    print(len(queue)) # quantas pages do site foram crawleadas
    
    return queue

In [7]:
# crawling preliminar

seeds = ["https://www.minhavida.com.br", "https://minutosaudavel.com.br", "https://medicoresponde.com.br",  
             "https://emagrecer.eco.br", "https://www.tuasaude.com", "https://www.mdsaude.com", 
             "http://portalms.saude.gov.br", "http://www.riocomsaude.rj.gov.br",
             "http://www.portalnatural.com", "http://portal.saude.pe.gov.br"] 

links_bsf = []

for link in seeds:
    print("Crawling", link, ":")
    links_bsf.append(crawl_bsf(link))  # lista de listas, para armazenar os links por site

Crawling https://www.minhavida.com.br :
1000
Crawling https://minutosaudavel.com.br :
2
Crawling https://medicoresponde.com.br :
1000
Crawling https://emagrecer.eco.br :
1000
Crawling https://www.tuasaude.com :
1000
Crawling https://www.mdsaude.com :
90
Crawling http://portalms.saude.gov.br :
1000
Crawling http://www.riocomsaude.rj.gov.br :
2
Crawling http://www.portalnatural.com :
1000
Crawling http://portal.saude.pe.gov.br :
1000


In [13]:
# crawling utilizado

# descartados "https://minutosaudavel.com.br" e "http://www.riocomsaude.rj.gov.br"

seeds = ["https://www.minhavida.com.br", "https://medicoresponde.com.br", "https://emagrecer.eco.br", 
             "https://www.tuasaude.com", "https://www.mdsaude.com", "http://portalms.saude.gov.br", 
             "http://www.portalnatural.com", "http://portal.saude.pe.gov.br"]

links_bsf = []

for link in seeds:
    print("Crawling", link, ":")
    links_bsf.append(crawl_bsf(link))  # lista de listas, para armazenar os links por site
    
links_bsf

Crawling https://www.minhavida.com.br :
1000
Crawling https://medicoresponde.com.br :
1000
Crawling https://emagrecer.eco.br :
1000
Crawling https://www.tuasaude.com :
1000
Crawling https://www.mdsaude.com :
90
Crawling http://portalms.saude.gov.br :
1000
Crawling http://www.portalnatural.com :
1000
Crawling http://portal.saude.pe.gov.br :
1000


[['https://www.minhavida.com.br',
  'https://www.minhavida.com.br/bem-estar/materias/33823-agir-por-impulso-pode-trazer-consequencias-desastrosas-para-a-saude-emocional',
  'https://www.minhavida.com.br/bem-estar/materias/33760-depressao-e-ansiedade-nao-sao-doencas-de-pessoas-fracas-sao-doencas-de-pessoas',
  'https://www.minhavida.com.br/alimentacao/listas/20614-20-dicas-para-emagrecer-rapido-e-com-saude',
  'https://www.minhavida.com.br/saude/temas/tercol',
  'https://www.minhavida.com.br/corporativo/quem-somos',
  'https://www.minhavida.com.br/beleza/tudo-sobre/16713-lipocavitacao-tratamento-para-gordura-localizada',
  'https://www.minhavida.com.br/alimentacao/tudo-sobre/17082-cha-de-hibisco-como-preparar-e-consumir-para-emagrecer',
  'https://www.minhavida.com.br/beleza',
  'https://www.minhavida.com.br/perguntas/câncer de mama',
  'https://www.minhavida.com.br/familia/materias/1177-20-dicas-para-ficar-de-bem-com-a-balanca-durante-a-gravidez',
  'https://www.minhavida.com.br/pergun

In [24]:
from file import dictionary_name as file

data = {'A': 'a', 'B': 'b', }

with open('file.txt','w') as file:
    file.write("dictionary_name = { \n")
    for k in sorted (data.keys()):
        file.write("'%s':'%s', \n" % (k, data[k]))
    file.write("}")

ModuleNotFoundError: No module named 'file'

In [19]:
import pandas as pd
# one_line_dict = exDict = {1:1, 2:2, 3:3}
# df = pd.DataFrame.from_dict([one_line_dict])
# df
# df.to_csv('file.txt', header=False, index=True, mode='a')

df = pd.DataFrame.from_dict(links_bsf)
df
df.to_csv('file.txt', header=False, index=True, mode='a')